In [ ]:
import pandas as pd
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from time import time
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/bullying detection/labeled_tweets.csv')

In [ ]:
df

,id,label,full_text
0,572342978255048705,Offensive,So Drasko just said he was impressed the girls...
1,572342978255048705,Offensive,So Drasko just said he was impressed the girls...
2,572341498827522049,Offensive,Drasko they didn't cook half a bird you idiot ...
3,572341498827522049,Offensive,Drasko they didn't cook half a bird you idiot ...
4,572340476503724032,Offensive,Hopefully someone cooks Drasko in the next ep ...
...,...,...,...
11085,576255143101882368,Non-offensive,RT @Transition: @freebsdgirl just so I’m clear...
11086,576481461424934912,Non-offensive,RT @Leonard_Delaney: @freebsdgirl I know! Holy...
11087,576607445692256256,Non-offensive,"FLOSS Weekly, open source projects, and paying..."
11088,572268344394399744,Non-offensive,RT @Kasparov63: My WSJ article on Boris Nemtso...


In [ ]:
df2=pd.read_csv('/content/drive/MyDrive/bullying detection/public_data_labeled.csv')

In [ ]:
df2

,label,full_text
0,Non-offensive,!!! RT @mayasolovely: As a woman you shouldn't...
1,Offensive,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,Offensive,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,Offensive,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,Offensive,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,Offensive,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,Non-offensive,"you've gone and broke the wrong heart baby, an..."
24780,Offensive,young buck wanna eat!!.. dat nigguh like I ain...
24781,Offensive,youu got wild bitches tellin you lies


In [ ]:
df_scraped=df
df_public=df2

In [ ]:
df_scraped.drop_duplicates(inplace = True)
df_scraped.drop('id', axis = 'columns', inplace = True)

df_public.drop_duplicates(inplace = True)

In [ ]:
df = pd.concat([df_scraped, df_public])

In [ ]:
df.shape

(35787, 2)

In [ ]:
df['label'] = df.label.map({'Offensive': 1, 'Non-offensive': 0})

In [ ]:
df

,label,full_text
0,1,So Drasko just said he was impressed the girls...
2,1,Drasko they didn't cook half a bird you idiot ...
4,1,Hopefully someone cooks Drasko in the next ep ...
6,1,of course you were born in serbia...you're as ...
7,1,These girls are the equivalent of the irritati...
...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,0,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies


In [ ]:
import nltk
nltk.download('punkt')
import string
import re
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
e=r.full_text.values
def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace
f=[]
for i in range(len(e)):
  f.append(clean_text(e[i]))

In [ ]:
tweets=np.array(f)
v=[]
for i in range(len(tweets)):
    print("tweet number processing:",i)
    words = nltk.word_tokenize(tweets[i])
    if(len(words)==0):
      v.append(i)
v

Streaming output truncated to the last 5000 lines.
tweet number processing: 30787
tweet number processing: 30788
tweet number processing: 30789
tweet number processing: 30790
tweet number processing: 30791
tweet number processing: 30792
tweet number processing: 30793
tweet number processing: 30794
tweet number processing: 30795
tweet number processing: 30796
tweet number processing: 30797
tweet number processing: 30798
tweet number processing: 30799
tweet number processing: 30800
tweet number processing: 30801
tweet number processing: 30802
tweet number processing: 30803
tweet number processing: 30804
tweet number processing: 30805
tweet number processing: 30806
tweet number processing: 30807
tweet number processing: 30808
tweet number processing: 30809
tweet number processing: 30810
tweet number processing: 30811
tweet number processing: 30812
tweet number processing: 30813
tweet number processing: 30814
tweet number processing: 30815
tweet number processing: 30816
tweet number proces

[]

In [ ]:
r=df
r1=r.label.values
df1=pd.DataFrame(r1,columns=['label'])
df1

,label
0,1
1,1
2,1
3,1
4,1
...,...
35782,1
35783,0
35784,1
35785,1


In [ ]:
df2=pd.DataFrame(tweets,columns=['tweet'])
final_df=pd.concat([df2,df1],axis=1)
final_df.shape

(35787, 2)

In [ ]:
import nltk
nltk.download('punkt')
import string
import re
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
final_df

,tweet,label
0,so drasko just said he was impressed the girls...,1
1,drasko they didnt cook half a bird you idiot mkr,1
2,hopefully someone cooks drasko in the next ep ...,1
3,of course you were born in serbiayoure as fuck...,1
4,these girls are the equivalent of the irritati...,1
...,...,...
35782,yous a muthafin lie lifeasking pearls coreyema...,1
35783,youve gone and broke the wrong heart baby and ...,0
35784,young buck wanna eat dat nigguh like i aint fu...,1
35785,youu got wild bitches tellin you lies,1


In [ ]:
import gensim 
from gensim.models import Word2Vec 
def vector_for_tweet(tweets):
  print("Vector for tweets calculating...")
  tweets_vec=[nltk.word_tokenize(i) for i in tweets]
  model=Word2Vec(tweets_vec,min_count=1,size=300)
  final_data=[]
  for i in range(len(tweets)):
    print("tweet number processing:",i)
    lst=[]
    words = nltk.word_tokenize(tweets[i])
    print("Length of words is:",len(words))
    max_arr = max_fun(words, model)
    min_arr = min_fun(words, model)
    avg_arr = avg_fun(words, model)
    lst.extend(max_arr)
    lst.extend(min_arr)
    lst.extend(avg_arr)
    final_data.append(lst)
  return final_data
def max_fun(words, model):
  print("max function")
  max_arr=[]
  for i in range(300):
    max = model[words[0]][i]
    for j in range(len(words)):
      if(model[words[j]][i]>max):
        max = model[words[j]][i]
    max_arr.append(max)
  print("max function completed for i",len(max_arr))
  return max_arr
def min_fun(words, model):
  print("min function")
  min_arr=[]
  for i in range(300):
    min = model[words[0]][i]
    for j in range(len(words)):
      if(model[words[j]][i]<min):
        min = model[words[j]][i]
    min_arr.append(min)
  print("min function completed for i",len(min_arr))
  return min_arr
def avg_fun(words, model):
  print("avg function")
  avg_arr=[]
  for i in range(300):
    count=0
    for j in range(len(words)):
      count+=model[words[j]][i]
    avg_arr.append(count/len(words))
  print("avg function completed for i",len(avg_arr))
  return avg_arr
tweets=final_df.tweet.values
final_vectors=vector_for_tweet(tweets)

Vector for tweets calculating...
tweet number processing: 0
Length of words is: 19
max function
max function completed for i 300
min function
min function completed for i 300
avg function
avg function completed for i 300
tweet number processing: 1
Length of words is: 10
max function
max function completed for i 300
min function
min function completed for i 300
avg function
avg function completed for i 300
tweet number processing: 2
Length of words is: 10
max function
max function completed for i 300
min function
min function completed for i 300
avg function
avg function completed for i 300
tweet number processing: 3
Length of words is: 14
max function
max function completed for i 300
min function
min function completed for i 300
avg function


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py

Streaming output truncated to the last 5000 lines.
tweet number processing: 35162
Length of words is: 16
max function
max function completed for i 300
min function
min function completed for i 300
avg function
avg function completed for i 300
tweet number processing: 35163
Length of words is: 10
max function
max function completed for i 300
min function
min function completed for i 300
avg function
avg function completed for i 300
tweet number processing: 35164
Length of words is: 14
max function
max function completed for i 300
min function
min function completed for i 300
avg function
avg function completed for i 300
tweet number processing: 35165
Length of words is: 12
max function
max function completed for i 300
min function
min function completed for i 300
avg function
avg function completed for i 300
tweet number processing: 35166
Length of words is: 6
max function
max function completed for i 300
min function
min function completed for i 300
avg function
avg function completed 

In [ ]:
len(final_vectors)

35787

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_vectors, 
                                                    final_df['label'], 
                                                    random_state=42)

In [ ]:
model_2=BaggingClassifier()
model_2.fit(X_train,y_train)
y_pred=model_2.predict(X_test)
accuracy_score(y_test,y_pred)

0.8510115122387393

In [ ]:
model_3=RandomForestClassifier()
model_3.fit(X_train,y_train)
y_pred=model_3.predict(X_test)
accuracy_score(y_test,y_pred)

0.8607354420476138

In [ ]:
model_4=SGDClassifier()
model_4.fit(X_train,y_train)
y_pred=model_4.predict(X_test)
accuracy_score(y_test,y_pred)

0.8368168101039455

In [ ]:
model_4=KNeighborsClassifier()
model_4.fit(X_train,y_train)
y_pred=model_4.predict(X_test)
accuracy_score(y_test,y_pred)

0.8164747960210126

In [ ]:
model_5=AdaBoostClassifier()
model_5.fit(X_train,y_train)
y_pred=model_5.predict(X_test)
accuracy_score(y_test,y_pred)

0.8571588241868783

In [ ]:
model_6=LinearSVC()
model_6.fit(X_train,y_train)
y_pred=model_6.predict(X_test)
accuracy_score(y_test,y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8635296747513133

In [ ]:
model_7=DecisionTreeClassifier()
model_7.fit(X_train,y_train)
y_pred=model_7.predict(X_test)
accuracy_score(y_test,y_pred)

0.8212808762713759

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['full_text'], 
                                                    df['label'], 
                                                    random_state=42)

In [ ]:
count_vector = CountVectorizer(stop_words = 'english', lowercase = True)

In [ ]:
training_data = count_vector.fit_transform(X_train)

In [ ]:
testing_data = count_vector.transform(X_test)

In [ ]:
model=AdaBoostClassifier()

In [ ]:
model.fit(training_data,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [ ]:
y_pred=model.predict(testing_data)

In [ ]:
accuracy_score(y_test, y_pred)

0.9075667821616185

In [ ]:
model_2=BaggingClassifier()

In [ ]:
model_2.fit(training_data,y_train)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

In [ ]:
y_pred2=model_2.predict(testing_data)

In [ ]:
accuracy_score(y_test, y_pred2)

0.9286911814015871

In [ ]:
a=['fuck you']

In [ ]:
 a=count_vector.transform(a)

In [ ]:
model_2.predict(a)

array([1])

In [ ]:
import pickle

In [ ]:

with open('/content/drive/MyDrive/bullying detection/bullying_model_file', 'wb') as fout:
    pickle.dump(model_2, fout)

In [ ]:
countvector_bullying_model = open('/content/drive/MyDrive/bullying detection/countvector_bullying_model_file', 'wb') 
pickle.dump(count_vector, countvector_bullying_model)

In [ ]:
with open('/content/drive/MyDrive/bullying detection/countvector_bullying_model_file', 'wb') as fout:
    pickle.dump(count_vector, fout)

In [ ]:
c1=0
c2=0
for i in y_pred2:
  if(i==1):
    c1=c1+1
  else:
    c2=c2+1

In [ ]:
c1

5614

In [ ]:
c2

3333